# ===================== OBJECTIVE =====================
Objective:
To design and implement a Generative Adversarial Network (GAN) that can generate realistic images
resembling the given dataset images. The goal is to understand the data distribution and evaluate
how well the model can synthesize new images.


In [1]:
# ===================== MOUNT DRIVE (IF NEEDED) =====================
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# ===================== IMPORT LIBRARIES =====================
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Reshape, Flatten, Dropout, Input, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from glob import glob
import tensorflow as tf
import cv2

In [3]:
# ===================== USER CONFIGURATION =====================
use_mnist = True  # Set to False if using custom images
image_folder = '/content/drive/MyDrive/your_image_folder'  # Only used if use_mnist is False
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100
save_interval = 5  # Save and display every 5 epochs

In [4]:
# ===================== LOAD DATA =====================
def load_images_from_folder(folder, img_shape):
    images = []
    for img_path in glob(os.path.join(folder, '*')):
        try:
            img = load_img(img_path, target_size=(img_shape[0], img_shape[1]), color_mode='grayscale' if img_shape[2] == 1 else 'rgb')
            img = img_to_array(img) / 127.5 - 1.0  # Normalize to [-1, 1]
            images.append(img)
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
    return np.array(images)

if use_mnist:
    (X_train, _), (_, _) = mnist.load_data()
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=-1)
else:
    X_train = load_images_from_folder(image_folder, img_shape)

print("Dataset loaded. Shape:", X_train.shape)

In [5]:
# ===================== BUILD GENERATOR =====================
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization())
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization())
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization())
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

In [6]:
# ===================== BUILD DISCRIMINATOR =====================
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [7]:
# ===================== BUILD & COMPILE GAN =====================
optimizer = Adam(0.0002, 0.5)

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

generator = build_generator()
z = Input(shape=(latent_dim,))
img = generator(z)
discriminator.trainable = False
valid = discriminator(img)

combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [8]:
# ===================== SAVE GENERATED IMAGES =====================
def save_images(epoch, generator, examples=16):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    gen_imgs = generator.predict(noise)

    gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale to [0,1]

    fig, axs = plt.subplots(4, 4, figsize=(5, 5))
    cnt = 0
    for i in range(4):
        for j in range(4):
            axs[i, j].imshow(gen_imgs[cnt].squeeze(), cmap='gray' if channels == 1 else None)
            axs[i, j].axis('off')
            cnt += 1
    plt.suptitle(f"Generated Images @ Epoch {epoch}", fontsize=12)
    plt.tight_layout()
    plt.show()

In [9]:
# ===================== TRAIN THE GAN =====================
epochs = 10000
batch_size = 64

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(1, epochs + 1):
    # Train Discriminator
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_imgs = X_train[idx]

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = combined.train_on_batch(noise, valid)

    # Print progress
    print(f"[Epoch {epoch}] [D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

    # Save and display every 5 epochs
    if epoch % save_interval == 0:
        save_images(epoch, generator)